# DCM Cost Model

## base data

In [ ]:
# Running this code will query a table in BigQuery and download
# the results to a Pandas DataFrame named `v_base`.
# Learn more here: https://cloud.google.com/bigquery/docs/visualize-jupyter

%%bigquery v_base

SELECT
  -- Delivery fields
  a.date,
  a.campaign,
  a.package_roadblock,
  a.package_id,
  a.placement_id,
  a.impressions,
  a.`KEY`, -- Using backticks as KEY is a reserved keyword
  a.`ad`,
  a.`click_rate`,
  a.`clicks`,
  a.`creative`,
  a.`media_cost`,
  a.`rich_media_video_completions`,
  a.`rich_media_video_plays`,
  a.`total_conversions`,

  -- Prisma metadata fields with aliases
  b.cost_method AS p_cost_method,
  -- Assuming the source column in 'b' is 'package_friendly'. If it's already 'p_package_friendly', use b.p_package_friendly
  b.p_package_friendly AS p_package_friendly,
  b.start_date AS p_start_date,

  -- [p_end_date] Effective end date with extended flight window for email packages (+60 days)
  CASE
    WHEN a.package_roadblock LIKE '%email%' THEN DATE_ADD(b.end_date, INTERVAL 60 DAY)
    ELSE b.end_date
  END AS p_end_date,

  b.total_days AS p_total_days,
  b.planned_daily_spend_pk     AS p_pkg_daily_planned_cost,
  b.planned_cost_pk            AS p_pkg_total_planned_cost,
  b.planned_daily_impressions_pk AS p_pkg_daily_planned_imps,
  b.planned_imps_pk            AS p_pkg_total_planned_imps,
  b.channel_group              AS p_channel_group,
  b.advertiser_name            AS p_advertiser_name,

  -- [flight_date_flag]   0 = in-flight, 1 = out-of-flight (based on the calculated p_end_date)
  CASE
    WHEN a.date BETWEEN b.start_date AND
        -- Use the p_end_date calculated above which includes the email extension
        (CASE WHEN a.package_roadblock LIKE '%email%' THEN DATE_ADD(b.end_date, INTERVAL 60 DAY) ELSE b.end_date END)
    THEN 0 ELSE 1
  END AS flight_date_flag,

  -- [flight_status_flag] 'live' if today is within the flight window (using calculated p_end_date), else 'ended'
  CASE
    WHEN CURRENT_DATE() BETWEEN b.start_date AND
        -- Use the p_end_date calculated above which includes the email extension
        (CASE WHEN a.package_roadblock LIKE '%email%' THEN DATE_ADD(b.end_date, INTERVAL 60 DAY) ELSE b.end_date END)
    THEN 'live' ELSE 'ended'
  END AS flight_status_flag,

  -- [rate_raw] Extracted numeric rate value from end of package_roadblock string
  REGEXP_EXTRACT(a.package_roadblock, r'(\d+\.?\d*)\D*$') AS rate_raw

FROM `giant-spoon-299605.data_model_2025.new_md` a
LEFT JOIN `looker-studio-pro-452620.20250327_data_model.prisma_porcessed` b
      ON a.package_id = b.package_id
WHERE a.impressions > 0
limit 100

In [16]:
# @title
v_base.head()

,date,campaign,package_roadblock,package_id,placement_id,impressions,KEY,ad,click_rate,clicks,...,p_total_days,p_pkg_daily_planned_cost,p_pkg_total_planned_cost,p_pkg_daily_planned_imps,p_pkg_total_planned_imps,p_channel_group,p_advertiser_name,flight_date_flag,flight_status_flag,rate_raw
0,2025-05-08,NYT Women's Suffrage Anniversary 1Q-3Q 2020,MM_Suffrage_NYT_NovPaidPost,None,None,2,2025-05-08MM_Suffrage_NYT_NovPaidPost_Footer_1x1,MM_Suffrage_NYT_NovPaidPost_Footer_1x1,0.0,0,...,<NA>,NaN,NaN,NaN,<NA>,None,None,1,ended,None
1,2025-05-08,MassMutualWLPP2022Q1,Package_MASS_MASS_MassMutualWLPP2022Q1_Display...,P2022Q1,P2022Q1,10,2025-05-08P1W8Y6S_FinsecaATFRectangleTextUnit2...,P1W8Y6S_FinsecaATFRectangleTextUnit2-16_IMG_Gu...,0.0,0,...,<NA>,NaN,NaN,NaN,<NA>,None,None,1,ended,4000.00
2,2025-05-08,MassMutual_2022B2B,Package_Standalone_MASS_BRAN_MassMutual2022B2B...,P21YFMG,P21YFMG,1,2025-05-08P21YFMG-FAIQSponsoredEmail-Nov-P21YF...,P21YFMG-FAIQSponsoredEmail-Nov-P21YFMG-Enterpr...,0.0,0,...,<NA>,NaN,NaN,NaN,<NA>,None,None,1,ended,9000.00
3,2025-05-08,MassMutualIMT2023,Package_MASS_MMSD_MassMutual2023IMT_Display_St...,P2CLF1L,P2CLF1M,1,2025-05-08P2CLF1L-ATFFSIMMSD-P2CLF1M-AnnuityCo...,P2CLF1L-ATFFSIMMSD-P2CLF1M-AnnuityConsideratio...,0.0,0,...,137,38.32,5250.0,0.0,0,display,MassMutual,1,ended,5250.00
4,2025-05-08,MassMutualQ22024,Package_MASS_BRAN_MassMutualQ22024_Other_Email...,P2PDHR2,P2PDHRC,15,2025-05-08P2PDHR2-DirectResponseSendAVJune1-P2...,P2PDHR2-DirectResponseSendAVJune1-P2PDHRC-News...,0.0,0,...,91,659.34,60000.0,98559.0,8968876,display,MassMutual,1,ended,60000.00


b